In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

from sklearn.tree import export_text
from sklearn import metrics
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df = pd.read_csv("all_tweets_v7.csv", index_col=0)
df.head(3)

/var/folders/z5/hjyvlbm54v1cpt9s019vbv9w0000gn/T/ipykernel_16428/685634843.py:1: DtypeWarning: Columns (14,15,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_tweets_v7.csv", index_col=0)


,author_id,context_annotations,conversation_id,created_at,edit_history_tweet_ids,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,reply_settings,text,author,withheld,geo,quote_count,retweet_count,like_count,reply_count,impression_count,media_keys,poll_ids,hashtags,urls,mentions,cashtags,annotations,is_retweet,gender_of_author,profession_of_author,hashtags_flattened,trend_topics,n_trend_topics,text_length,has_media,has_hashtags,has_mentions,is_reply,tweet_type,ratio_like,ratio_retweet,majority_lang,english,universal,eng_astroturf,eng_fake_follower,eng_financial,eng_other,eng_overall,eng_self_declared,eng_spammer,uni_astroturf,uni_fake_follower,uni_financial,uni_other,uni_overall,uni_self_declared,uni_spammer,verified_author,followers_count_author,following_count_author,tweet_count_author,age_of_account_in_days_author,sentiment,created_at_day_of_week,created_at_month_of_year,created_at_time_of_day_in_seconds,reply_to_tweet_ratio,retweet_to_tweet_ratio,average_tweets_of_author_per_day
0,1449804331142811655,NaN,1617597872803041280,2023-01-23 18:59:21,[1617597872803041280],1617597872803041280,NaN,tr,False,NaN,everyone,"Kararlarındaki temel dayanak, hukukun evrensel...",Goksun_KHK,NaN,NaN,0,2,1,0,82,NaN,NaN,"[{'start': 95, 'end': 120, 'tag': 'OhalKomisyo...",NaN,NaN,NaN,NaN,0,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur,OhalKomisyonuHukuksuzdur,1,120,0,1,0,0,no_reply_and_no_retweet,0.012195,0.024390,tr,0.874858,0.847463,0.13,0.51,0.22,0.91,0.91,0.08,0.25,0.12,0.44,0.1,0.87,0.87,0.19,0.16,False,1107.0,174.0,2962.0,470.0,0.994189,0,1,68361,0.013817,0.435233,1.231915
1,1449804331142811655,NaN,1617597646339702823,2023-01-23 18:58:27,[1617597646339702823],1617597646339702823,NaN,fr,False,NaN,everyone,#OhalKomisyonuHukuksuzdur https://t.co/3aUr5MZWPU,Goksun_KHK,NaN,NaN,0,1,0,0,52,['3_1617597636604723228'],NaN,"[{'start': 0, 'end': 25, 'tag': 'OhalKomisyonu...","[{'start': 26, 'end': 49, 'url': 'https://t.co...",NaN,NaN,NaN,0,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur,OhalKomisyonuHukuksuzdur,1,49,1,1,0,0,no_reply_and_no_retweet,0.000000,0.019231,tr,0.874858,0.847463,0.13,0.51,0.22,0.91,0.91,0.08,0.25,0.12,0.44,0.1,0.87,0.87,0.19,0.16,False,1107.0,174.0,2962.0,470.0,0.733746,0,1,68307,0.013817,0.435233,1.231915
2,1449804331142811655,NaN,1617597571903676417,2023-01-23 18:58:09,[1617597571903676417],1617597571903676417,NaN,en,False,[<ReferencedTweet id=1617584897383714816 type=...,everyone,@Yozgat_KHK: #OhalKomisyonuHukuksuzdur\nÇünkü ...,Goksun_KHK,NaN,NaN,0,48,0,0,0,NaN,NaN,"[{'start': 16, 'end': 41, 'tag': 'OhalKomisyon...",NaN,"[{'start': 3, 'end': 14, 'username': 'Yozgat_K...",NaN,NaN,1,ORG,NOT AVAILABLE,OhalKomisyonuHukuksuzdur,OhalKomisyonuHukuksuzdur,1,99,0,1,1,0,only_retweet,NaN,inf,tr,0.874858,0.847463,0.13,0.51,0.22,0.91,0.91,0.08,0.25,0.12,0.44,0.1,0.87,0.87,0.19,0.16,False,1107.0,174.0,2962.0,470.0,0.000000,0,1,68289,0.013817,0.435233,1.231915


# Preprocessing

## Column Types

In [108]:
irrelevant_columns = [
    "author_id", 
    "conversation_id",
    "created_at",
    "edit_history_tweet_ids",  # all of them with len of 1 and includes id
    "id",  # TODO: investigate when it is different than conversation id
    "in_reply_to_user_id",
    "referenced_tweets",
    "text",  # TODO: investigate how to integrate it,
    "author",  # TODO: investigate how to integrate it
    "hashtags",
    "mentions",
    "hashtags_flattened",
    "trend_topics",
    "media_keys",  # TODO: investigate how to incorporate more
]

is_exist_columns = [
    "geo", 
    "context_annotations",  # TODO: investigate how we can interpret it
    "withheld",  # TODO: investigate how to integrate it
    "poll_ids",
    "annotations",  # TODO: investigate how we can interpret it
]

len_columns = [
    "urls",
    "cashtags",
]

categorical_columns = [
    "lang",
    "reply_settings",
    "gender_of_author",
    "profession_of_author",
    "tweet_type",
    "majority_lang",
    "created_at_day_of_week",
    "created_at_month_of_year"
]

binary_columns = [
    "is_retweet",
    "has_media",
    "has_hashtags",
    "is_reply",
    "verified_author",
]

target_columns = [
    "quote_count",
    "retweet_count",
    "like_count",
    "reply_count",
    "impression_count",
    "ratio_like",
    "ratio_retweet",
]

numerical_columns = [
    "n_trend_topics",
    "text_length",
    
    # Bot score features
    "english",
    "universal",
    "eng_astroturf",
    "eng_fake_follower",
    "eng_financial",
    "eng_other",
    "eng_overall",
    "eng_self_declared",
    "eng_spammer",
    "uni_astroturf",
    "uni_fake_follower",
    "uni_financial",
    "uni_other",
    "uni_overall",
    "uni_self_declared",
    "uni_spammer",
    
    "followers_count_author",
    "following_count_author",
    "tweet_count_author",
    "age_of_account_in_days_author",
    "sentiment",
    
    "created_at_time_of_day_in_seconds",  # TODO Better representation for time features
    "reply_to_tweet_ratio",
    "retweet_to_tweet_ratio",
    "average_tweets_of_author_per_day",
]



## Preprocessing columns

In [127]:
preprocessed_df = df.copy()

In [128]:
preprocessed_df.drop(columns=irrelevant_columns, inplace=True)

In [129]:
preprocessed_df[is_exist_columns] = preprocessed_df[is_exist_columns].isna()
preprocessed_df[is_exist_columns].head()

,geo,context_annotations,withheld,media_keys,poll_ids,annotations
0,True,True,True,True,True,True
1,True,True,True,False,True,True
2,True,True,True,True,True,True
3,True,True,True,False,True,True
4,True,True,True,True,True,True


In [130]:
preprocessed_df[len_columns].head()

,urls,cashtags
0,NaN,NaN
1,"[{'start': 26, 'end': 49, 'url': 'https://t.co...",NaN
2,NaN,NaN
3,"[{'start': 26, 'end': 49, 'url': 'https://t.co...",NaN
4,NaN,NaN


In [131]:
for c in len_columns:  # TODO find a better method instead of for loop
    preprocessed_df[c] = preprocessed_df[c].apply(lambda x: len(eval(x)) if type(x) != float else 0)

assert preprocessed_df[len_columns][preprocessed_df[len_columns].isna().any(axis=1)].shape[0] == 0 
preprocessed_df[len_columns].head()

,urls,cashtags
0,0,0
1,1,0
2,0,0
3,1,0
4,0,0


In [132]:
preprocessed_df = pd.get_dummies(preprocessed_df, columns=categorical_columns, drop_first=True)
preprocessed_df.head()

,context_annotations,possibly_sensitive,withheld,geo,quote_count,retweet_count,like_count,reply_count,impression_count,media_keys,poll_ids,urls,cashtags,annotations,is_retweet,n_trend_topics,text_length,has_media,has_hashtags,has_mentions,is_reply,ratio_like,ratio_retweet,english,universal,eng_astroturf,eng_fake_follower,eng_financial,eng_other,eng_overall,eng_self_declared,eng_spammer,uni_astroturf,uni_fake_follower,uni_financial,uni_other,uni_overall,uni_self_declared,uni_spammer,verified_author,followers_count_author,following_count_author,tweet_count_author,age_of_account_in_days_author,sentiment,created_at_time_of_day_in_seconds,reply_to_tweet_ratio,retweet_to_tweet_ratio,average_tweets_of_author_per_day,lang_art,lang_ca,lang_ckb,lang_cs,lang_cy,lang_da,lang_de,lang_el,lang_en,lang_es,lang_et,lang_eu,lang_fa,lang_fi,lang_fr,lang_hi,lang_ht,lang_hu,lang_hy,lang_in,lang_is,lang_it,lang_iw,lang_ja,lang_lt,lang_lv,lang_nl,lang_no,lang_pl,lang_pt,lang_qam,lang_qht,lang_qme,lang_qst,lang_ro,lang_ru,lang_sl,lang_sv,lang_tl,lang_tr,lang_ug,lang_uk,lang_und,lang_vi,lang_zh,lang_zxx,reply_settings_following,reply_settings_mentionedUsers,gender_of_author_M,gender_of_author_ORG,gender_of_author_UNKNOWN,profession_of_author_ARTIST,profession_of_author_BASKETBALL PLAYER,profession_of_author_DOCTOR,profession_of_author_JOURNALIST,profession_of_author_LAWYER,profession_of_author_NOT AVAILABLE,profession_of_author_POLITICIAN,profession_of_author_TEACHER,profession_of_author_UNKNOWN,profession_of_author_WRITER,tweet_type_only_reply,tweet_type_only_retweet,majority_lang_tr,created_at_day_of_week_1,created_at_day_of_week_2,created_at_day_of_week_3,created_at_day_of_week_4,created_at_day_of_week_5,created_at_day_of_week_6,created_at_month_of_year_2,created_at_month_of_year_3,created_at_month_of_year_4,created_at_month_of_year_5,created_at_month_of_year_6,created_at_month_of_year_7,created_at_month_of_year_8,created_at_month_of_year_9,created_at_month_of_year_10,created_at_month_of_year_11,created_at_month_of_year_12
0,True,False,True,True,0,2,1,0,82,True,True,0,0,True,0,1,120,0,1,0,0,0.012195,0.024390,0.874858,0.847463,0.13,0.51,0.22,0.91,0.91,0.08,0.25,0.12,0.44,0.1,0.87,0.87,0.19,0.16,False,1107.0,174.0,2962.0,470.0,0.994189,68361,0.013817,0.435233,1.231915,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,True,False,True,True,0,1,0,0,52,False,True,1,0,True,0,1,49,1,1,0,0,0.000000,0.019231,0.874858,0.847463,0.13,0.51,0.22,0.91,0.91,0.08,0.25,0.12,0.44,0.1,0.87,0.87,0.19,0.16,False,1107.0,174.0,2962.0,470.0,0.733746,68307,0.013817,0.435233,1.231915,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,True,False,True,True,0,48,0,0,0,True,True,0,0,True,1,1,99,0,1,1,0,NaN,inf,0.874858,0.847463,0.13,0.51,0.22,0.91,0.91,0.08,0.25,0.12,0.44,0.1,0.87,0.87,0.19,0.16,False,1107.0,174.0,2962.0,470.0,0.000000,68289,0.013817,0.435233,1.231915,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,True,False,True,True,1,1,1,0,86,False,True,1,0,True,0,1,49,1,1,0,0,0.011628,0.011628,0.874858,0.847463,0.13,0.51,0.22,0.91,0.91,0.08,0.25,0.12,0.44,0.1,0.87,0.87,0.19,0.16,False,1107.0,174.0,2962.0,470.0,-0.599463,68282,0.013817,0.435233,1.231915,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,True,False,True,True,0,4,0,0,0,True,True,0,0,True,1,1,82,0,1,1,0,NaN,inf,0.874858,0.847463,0.13,0.51,0.22,0.91,0.91,0.08,0.25,0.12,0.44,0.1,0.87,0.87,0.19,0.16,False,1107.0,174.0,2962.0,470.0,0.900369,68266,0.013817,0.435233,1.231915,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [143]:
preprocessed_df.sentiment.fillna(0, inplace=True) # TODO: include a new variable to indicate this imputation
preprocessed_df.reply_to_tweet_ratio.fillna(0, inplace=True)  # TODO: investigate why this happens

In [144]:
preprocessed_df.dtypes

context_annotations                          bool
possibly_sensitive                           bool
withheld                                     bool
geo                                          bool
quote_count                                 int64
retweet_count                               int64
like_count                                  int64
reply_count                                 int64
impression_count                            int64
media_keys                                   bool
poll_ids                                     bool
urls                                        int64
cashtags                                    int64
annotations                                  bool
is_retweet                                  int64
n_trend_topics                              int64
text_length                                 int64
has_media                                   int64
has_hashtags                                int64
has_mentions                                int64


In [154]:
# Columns with missing values
for i in preprocessed_df[preprocessed_df.isna().any(axis=1)].isna().any().items():
    if i[1]:
        print(i[0])

ratio_like
ratio_retweet


# Modelling

In [155]:
X = preprocessed_df[[c for c in preprocessed_df.columns if c not in target_columns ]]
Y = preprocessed_df[[c for c in preprocessed_df.columns if c in target_columns ]]

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(269277, 123) (89759, 123) (269277, 7) (89759, 7)


In [190]:
def fit_and_evaluate(model_name, model_params):
    model_class, params = model_params
    model = model_class(**params)
    print("Training the model")
    model.fit(x_train, y_train.like_count)
    y_test_predicted = model.predict(x_test)
    
    print("Predicting with the trained model")

    mae = metrics.mean_absolute_error(y_test.like_count, y_test_predicted)
    mse = metrics.mean_squared_error(y_test.like_count, y_test_predicted)
    r2 = metrics.r2_score(y_test.like_count, y_test_predicted)

    print(f"The {model_name} model performance for testing set")
    print("--------------------------------------")
    print(f'MAE is {mae}')
    print(f'MSE is {mse}')
    print(f'R2 score is {r2}')
    return model

In [191]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

model_classes = {
    "decision_tree": (DecisionTreeRegressor, {}),
    "linear_regression": (LinearRegression, {}),
    "random_forest": (RandomForestRegressor, {"n_estimators": 500, "random_state": 0, "n_jobs": -1}), 
}

In [192]:
models = {}
for model_name, params in model_classes.items():
    models[model_name] = fit_and_evaluate(model_name, params)

Training the model
Predicting with the trained model
The <class 'sklearn.tree._classes.DecisionTreeRegressor'> model performance for testing set
--------------------------------------
MAE is 64.08714260779793
MSE is 487203.5234464263
R2 score is 0.039486494466661304
Training the model
Predicting with the trained model
The <class 'sklearn.linear_model._base.LinearRegression'> model performance for testing set
--------------------------------------
MAE is 168.26761352967395
MSE is 418526.6989196398
R2 score is 0.17488169236771933
Training the model
Predicting with the trained model
The <class 'sklearn.ensemble._forest.RandomForestRegressor'> model performance for testing set
--------------------------------------
MAE is 49.509838882637574
MSE is 268627.31464946916
R2 score is 0.47040579294098406


# TODO

1. Outlier Detection
2. Different model architectures
3. Shapley values for black-box models